## Set 5
## 3. Word2Vec **Principles**

#### Preparation


The following function may be useful for loading the necessary data.

In [2]:
import requests

url_dict = {
    'dr_seuss.txt': 'https://caltech-cs155.s3.us-east-2.amazonaws.com/sets/set5/data/dr_seuss.txt',
    'P3CHelpers.py': 'https://caltech-cs155.s3.us-east-2.amazonaws.com/sets/set5/code/P3CHelpers.py'
}

def download_file(file_path):
    url = url_dict[file_path]
    print('Start downloading...')
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 1024 * 1024):
                f.write(chunk)
    print('Complete')

In [3]:
# Download files
download_file('dr_seuss.txt')
download_file('P3CHelpers.py')

Start downloading...
Complete
Start downloading...
Complete


In [4]:
import numpy as np
from P3CHelpers import *
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset

#### Problem D:
Fill in the generate_traindata and find_most_similar_pairs functions.

In [5]:
def get_word_repr(word_to_index, word):
    """
    Returns one-hot-encoded feature representation of the specified word given
    a dictionary mapping words to their one-hot-encoded index.

    Arguments:
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        word:          Word whose feature representation we wish to compute.

    Returns:
        feature_representation:     Feature representation of the passed-in word.
    """
    unique_words = word_to_index.keys()
    # Return a vector that's zero everywhere besides the index corresponding to <word>
    feature_representation = np.zeros(len(unique_words))
    feature_representation[word_to_index[word]] = 1
    return feature_representation

def generate_traindata(word_list, word_to_index, window_size=4):
    """
    Generates training data for Skipgram model.

    Arguments:
        word_list:     Sequential list of words (strings).
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        window_size:   Size of Skipgram window. Defaults to 2
                       (use the default value when running your code).

    Returns:
        (trainX, trainY):     A pair of matrices (trainX, trainY) containing training
                              points (one-hot-encoded vectors) and their corresponding output_word
                              (also one-hot-encoded vectors)


    """
    trainX = []
    trainY = []

    ##############################################################
    # TODO: Implement this function, populating trainX and trainY
    ##############################################################
    for i in range(len(word_list)):
      start_index = max(i-window_size, 0)
      end_index = min(i + window_size + 1, len(word_list))
      for n in range(start_index, end_index):
        if n == i:
          continue
        trainX.append(get_word_repr(word_to_index, word_list[i]))
        trainY.append(get_word_repr(word_to_index, word_list[n]))


    return np.array(trainX), np.array(trainY)

In [8]:
def find_most_similar_pairs(filename, num_latent_factors):
    """
    Find the most similar pairs from the word embeddings computed from
    a body of text

    Arguments:
        filename:           Text file to read and train embeddings from
        num_latent_factors: The number of latent factors / the size of the embedding
    """
    # Load in a list of words from the specified file; remove non-alphanumeric characters
    # and make all chars lowercase.
    sample_text = load_word_list(filename)

    # Create word dictionary
    word_to_index = generate_onehot_dict(sample_text)
    print("Textfile contains %s unique words"%len(word_to_index))
    # Create training data
    trainX, trainY = generate_traindata(sample_text, word_to_index)

    ##############################################################
    # TODO: 1) Create and train model in Pytorch.
    ##############################################################


    # vocab_size = number of unique words in our text file. Will be useful
    # when adding layers to your neural network
    vocab_size = len(word_to_index)

    model = nn.Sequential(
      nn.Linear(vocab_size, 10),
      nn.Linear(10, vocab_size),
      nn.Softmax(dim = None)
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss()

    x = torch.from_numpy(trainX).float()
    y = torch.from_numpy(trainY).float()
    data = TensorDataset(x, y)

    train_loader = torch.utils.data.DataLoader(data, batch_size=32, shuffle=True)

    model.train()

    for epoch in range(5):
        for batch_idx, (data, target) in enumerate(train_loader):
            # Erase accumulated gradients
            optimizer.zero_grad()

            # Forward pass
            output = model(data)

            # Calculate loss
            loss = loss_fn(output, target)

            # Backward pass
            loss.backward()

            # Weight update
            optimizer.step()

    ##############################################################
    # TODO: 2) Extract weights for hidden layer
    ##############################################################

    # set weights variable below
    print(model)
    weights = model[1].weight.detach().numpy()

    print(model[0].weight.detach().numpy().shape)
    print(weights.shape)

    # Find and print most similar pairs
    similar_pairs = most_similar_pairs(weights, word_to_index)
    for pair in similar_pairs[:30]:
        print(pair)

### Problem E-H:
Run your model on drseuss.txt and answer questions from E through H.

In [9]:
find_most_similar_pairs('dr_seuss.txt', 10)

Textfile contains 308 unique words
Sequential(
  (0): Linear(in_features=308, out_features=10, bias=True)
  (1): Linear(in_features=10, out_features=308, bias=True)
  (2): Softmax(dim=None)
)
(10, 308)
(308, 10)
Pair(gold, quiet), Similarity: 0.9950512
Pair(quiet, gold), Similarity: 0.9950512
Pair(too, over), Similarity: 0.9938831
Pair(over, too), Similarity: 0.9938831
Pair(mother, fear), Similarity: 0.99288285
Pair(fear, mother), Similarity: 0.99288285
Pair(dog, time), Similarity: 0.99229324
Pair(time, dog), Similarity: 0.99229324
Pair(bet, grow), Similarity: 0.9916504
Pair(grow, bet), Similarity: 0.9916504
Pair(where, park), Similarity: 0.9913765
Pair(park, where), Similarity: 0.9913765
Pair(dad, where), Similarity: 0.99134386
Pair(cats, live), Similarity: 0.991159
Pair(live, cats), Similarity: 0.991159
Pair(yellow, ish), Similarity: 0.9909122
Pair(ish, yellow), Similarity: 0.9909122
Pair(cow, where), Similarity: 0.9907321
Pair(seven, just), Similarity: 0.99053955
Pair(just, seven), 